In [ ]:
from booknlp.booknlp import BookNLP

model_params={
		"pipeline":"entity,quote,supersense,event,coref", 
		"model":"big"
	}
	
booknlp=BookNLP("en", model_params)




In [3]:
# Input file to process
input_file="a_little_princess.txt"

# Output directory to store resulting files in
output_directory="./output_dir/princess/"

# File within this directory will be named ${book_id}.entities, ${book_id}.tokens, etc.
book_id="princess"

booknlp.process(input_file, output_directory, book_id)


--- spacy: 7.329 seconds ---
--- entities: 42.074 seconds ---
--- quotes: 0.043 seconds ---
--- attribution: 86.122 seconds ---
--- name coref: 0.046 seconds ---
--- coref: 49.892 seconds ---
--- TOTAL (excl. startup): 185.642 seconds ---, 81696 words


In [17]:
import json
from collections import Counter

def proc(filename):
    with open(filename) as file:
        data=json.load(file)
    return data

data=proc("./output_dir/bartleby/bartleby.book")

In [18]:
def get_counter_from_dependency_list(dep_list):
    counter=Counter()
    for token in dep_list:
        term=token["w"]
        tokenGlobalIndex=token["i"]
        counter[term]+=1
    return counter

for character in data["characters"]:

    agentList=character["agent"]
    patientList=character["patient"]
    possList=character["poss"]
    modList=character["mod"]

    character_id=character["id"]
    count=character["count"]

    referential_gender_distribution=referential_gender_prediction="unknown"

    if character["g"] is not None and character["g"] != "unknown":
        referential_gender_distribution=character["g"]["inference"]
        referential_gender=character["g"]["argmax"]

    mentions=character["mentions"]
    proper_mentions=mentions["proper"]
    max_proper_mention=""

    # just print out information about named characters
    if len(mentions["proper"]) > 0:
        max_proper_mention=mentions["proper"][0]["n"]

        print(character_id, count, max_proper_mention, referential_gender)

        print()
        printTop=100
        for k, v in get_counter_from_dependency_list(possList).most_common(printTop):
            print("\tposs\t%s %s" % (v,k))
        print()
        for k, v in get_counter_from_dependency_list(agentList).most_common(printTop):
            print("\tagent\t%s %s" % (v,k))
        print()
        for k, v in get_counter_from_dependency_list(patientList).most_common(printTop):
            print("\tpatient\t%s %s" % (v,k))
        print()
        for k, v in get_counter_from_dependency_list(modList).most_common(printTop):
            print("\tmod\t%s %s" % (v,k))
        print()

27 522 Bartleby he/him/his

	poss	5 desk
	poss	5 hermitage
	poss	5 screen
	poss	4 eyes
	poss	4 window
	poss	3 papers
	poss	3 ways
	poss	2 services
	poss	2 writing
	poss	2 privacy
	poss	2 face
	poss	2 manner
	poss	2 conduct
	poss	2 eccentricities
	poss	2 home
	poss	2 history
	poss	2 place
	poss	1 case
	poss	1 youth
	poss	1 qualifications
	poss	1 application
	poss	1 copy
	poss	1 retreat
	poss	1 eye
	poss	1 document
	poss	1 legs
	poss	1 copies
	poss	1 request
	poss	1 decision
	poss	1 business
	poss	1 evinces
	poss	1 willfulness
	poss	1 freedom
	poss	1 industry
	poss	1 stillness
	poss	1 unalterableness
	poss	1 honesty
	poss	1 hands
	poss	1 part
	poss	1 finger
	poss	1 _
	poss	1 mildness
	poss	1 clerk
	poss	1 premises
	poss	1 sleeves
	poss	1 poverty
	poss	1 solitude
	poss	1 etc
	poss	1 .
	poss	1 glance
	poss	1 answer
	poss	1 countenance
	poss	1 perverseness
	poss	1 behavior
	poss	1 reply
	poss	1 tongue
	poss	1 revery
	poss	1 diligence
	poss	1 stay
	poss	1 vision
	poss	1 account
	poss	1 shoul

In [19]:
import pandas as pd

df_tokens = pd.read_csv("./output_dir/bartleby/bartleby.tokens", delimiter="\t")
df_entities = pd.read_csv("./output_dir/bartleby/bartleby.entities", delimiter="\t")

#create dictionary with the shape [sentence_ID][token_ID_within_sentence][word]
sentence_and_token_ID_to_word_dict = {}
for character in data["characters"]:
    character_id=character["id"]
    mentions=character["mentions"]
    proper_mentions=mentions["proper"]
    if len(mentions["proper"]) > 0:
        max_proper_mention=mentions["proper"][0]["n"]
    else:
        max_proper_mention = character_id

    unique_start_token_ids = df_entities.loc[df_entities['COREF'] == character_id, 'start_token']
    #print(unique_start_token_ids)
    sentence_ids = df_tokens[df_tokens['token_ID_within_document'].isin(unique_start_token_ids)][['token_ID_within_document', 'token_ID_within_sentence', 'sentence_ID']]
    sentence_ids = sentence_ids.reset_index(drop=True)

    for index, row in sentence_ids.iterrows():
        sentence_ID = row['sentence_ID']
        token_ID = row['token_ID_within_sentence']
        combined_sentence_and_token_ID = str(sentence_ID) +":"+ str(token_ID)
        sentence_and_token_ID_to_word_dict[combined_sentence_and_token_ID] = max_proper_mention
    

In [25]:
character = data["characters"][1]
character_id=character["id"]

#get all sentencs where COREF = character_id
unique_start_token_ids = df_entities.loc[df_entities['COREF'] == character_id, 'start_token']
print(unique_start_token_ids)
sentence_ids = df_tokens[df_tokens['token_ID_within_document'].isin(unique_start_token_ids)][['token_ID_within_document', 'token_ID_within_sentence', 'sentence_ID']]
sentence_ids = sentence_ids.reset_index(drop=True)
print(sentence_ids)

0           5
12        107
17        163
19        177
36        304
        ...  
2456    17354
2458    17363
2468    17483
2472    17499
2483    17670
Name: start_token, Length: 522, dtype: int64
     token_ID_within_document  token_ID_within_sentence  sentence_ID
0                           5                         5            0
1                         107                         2            3
2                         163                         0            4
3                         177                        14            4
4                         304                        17            8
..                        ...                       ...          ...
290                     17354                        35          791
291                     17363                        44          791
292                     17483                         6          795
293                     17499                        22          795
294                     17670             

In [26]:
from allennlp.predictors.predictor import Predictor
SRL_MODEL_PATH = "srl.tar.gz"
predictor_srl = Predictor.from_path(SRL_MODEL_PATH)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [27]:
import utils
import spacy
nlp = spacy.load("en_core_web_sm")

In [28]:
def check_for_arg(n, arg_name, dict_name, sentence_ID):
    if arg_name in n["tags"]:
        index = n['tags'].index(arg_name)
        if str(sentence_ID)+":"+str(index) in dict_name.keys():
            return dict_name[str(sentence_ID)+":"+str(index)]
        else:
            return None
    else:
        return None
    


In [29]:
columns = ['sentence_ID', 'verb', 'agent', 'patient']
srl_results_df = pd.DataFrame(columns=columns)

# for each mention in our coreference cluster do the following
for index, row in sentence_ids.iterrows():
    token_ID_within_document = row['token_ID_within_document']
    #word = df_tokens.loc[df_tokens['token_ID_within_document'] == token_ID_within_document, 'word'].values[0]

    sentence_ID = df_tokens.loc[df_tokens['token_ID_within_document'] == token_ID_within_document, 'sentence_ID'].values[0]
    words_list = df_tokens.loc[df_tokens['sentence_ID'] == sentence_ID, 'word'].tolist()
    token_ID_within_sentence = row['token_ID_within_sentence']
    sentence_for_srl = ' '.join(words_list)
    predictions_srl = predictor_srl.predict(sentence_for_srl)

    for n in predictions_srl['verbs']:
        if token_ID_within_sentence < len(n['tags']):
            tag = n['tags'][token_ID_within_sentence]
            if "B-ARG0" in tag:

                word = check_for_arg(n, "B-ARG1", sentence_and_token_ID_to_word_dict, sentence_ID)
                if word != None:
                    new_row = pd.DataFrame({'sentence_ID': [sentence_ID], 'verb': [utils.lemmatize(n['verb'], nlp)], 'agent': ["protag"], 'patient': [word]})
                    srl_results_df = pd.concat([srl_results_df, new_row], ignore_index=True)
            elif "B-ARG1" in tag:
                word = check_for_arg(n, "B-ARG0", sentence_and_token_ID_to_word_dict, sentence_ID)
                if word != None:
                    new_row = pd.DataFrame({'sentence_ID': [sentence_ID], 'verb': [utils.lemmatize(n['verb'], nlp)], 'agent': [word], 'patient': ["protag"]})
                    srl_results_df = pd.concat([srl_results_df, new_row], ignore_index=True)

srl_results_df.to_csv('srl_output_bartleby.csv', index=False)
del srl_results_df